In [1]:
import pybullet as p
import numpy as np
import time
from environment.utilities import setup_sisbot
from pathlib import Path

pybullet build time: Nov 28 2023 23:45:17


In [2]:
p.connect(p.GUI)
robot_id = p.loadURDF("environment/urdf/ur5_robotiq_140.urdf", useFixedBase=1)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-S)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RPL-S)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No 

In [3]:
joints, controlGripper, controlJoints, mimicParentName = setup_sisbot(p, robot_id, "140")

def reset_robot():
    user_parameters = (-1.5690622952052096, -1.5446774605904932, 1.343946009733127, -1.3708613585093699,
                        -1.5707970583733368, 0.0009377758247187636, 0.085)
    for _ in range(60):
        for i, name in enumerate(controlJoints):
            if i == 6:
                controlGripper(controlMode=p.POSITION_CONTROL, targetPosition=user_parameters[i])
                break
            joint = joints[name]
            # control robot joints
            p.setJointMotorControl2(robot_id, joint.id, p.POSITION_CONTROL,
                                    targetPosition=user_parameters[i], force=joint.maxForce,
                                    maxVelocity=joint.maxVelocity)
            p.stepSimulation()

reset_robot()
for _ in range(300):
    p.stepSimulation()

In [37]:
reset_robot()

In [38]:
def generate_linear_path(start_pos, end_pos, steps=100):
    return np.linspace(start_pos, end_pos, steps)

# Identify the end effector link index
end_effector_link_index = 7  # Replace with your end-effector link index

# Get the initial position and orientation of the end effector
start_position = p.getLinkState(robot_id, end_effector_link_index)[0]

In [6]:

# Generate a linear path from the start to the end position
end_position = np.array(start_position) + (np.random.random(size=3)-0.5)*0.25
end_position = end_position.tolist()
path = generate_linear_path(start_position, end_position, steps=100)

debug_id = p.addUserDebugLine(start_position, end_position, [1,0,0])


In [7]:

# # Move the robot's end effector along the linear path
# for target_pos in path:
#     # Compute the joint angles using inverse kinematics
#     joint_poses = p.calculateInverseKinematics(robot_id, end_effector_link_index, target_pos)

#     # Set the robot's joint motors to the calculated joint positions
#     for i in range(len(joint_poses)):
#         if p.getJointInfo(robot_id, i)[1].decode("utf-8") not in controlJoints:
#             continue
#         p.setJointMotorControl2(bodyIndex=robot_id, 
#                                 jointIndex=i, 
#                                 controlMode=p.POSITION_CONTROL, 
#                                 targetPosition=joint_poses[i])

#     # Step the simulation
#     p.stepSimulation()
#     time.sleep(1./60.)


# time.sleep(2)
# start_position = p.getLinkState(robot_id, end_effector_link_index)[0]

In [8]:
prev_target_pos = path[0]
for target_pos in path[1:]:
    p.addUserDebugLine(prev_target_pos, target_pos, [0,1,0])

In [52]:
def get_joint_name(robot_id, joint_id: int) -> str:
    return p.getJointInfo(robot_id, joint_id)[1].decode('utf-8')

reset_robot()

def linear_move(end_point):
    current_point = p.getLinkState(robot_id, end_effector_link_index)[0]
    current_orn = p.getLinkState(robot_id, end_effector_link_index)[1]

    path = np.linspace(current_point, end_point, 100)
    for pidx, target_pos in enumerate(path):
        joint_poses = p.calculateInverseKinematics(robot_id, end_effector_link_index, target_pos, current_orn, maxNumIterations=100)

        # Set Joint motor control of all control joints except the gripper
        for i, name in enumerate(controlJoints[:-1]):
            joint = joints[name]
            pose = joint_poses[i]
            # control robot end-effector
            p.setJointMotorControl2(robot_id, joint.id, p.POSITION_CONTROL,
                                    targetPosition=pose, force=joint.maxForce,
                                    maxVelocity=joint.maxVelocity)

        p.stepSimulation()
        time.sleep(1./240)

c = p.getLinkState(robot_id, end_effector_link_index)[0]
p0, p1, p2 = [c[0]-0.2, c[1], c[2]], [c[0], c[1], c[2] -0.2], [c[0]+0.2, c[1], c[2]]
linear_move(p0)
linear_move(p1)
linear_move(p2)
linear_move(c)


: 

In [48]:
p.removeAllUserDebugItems()